In [1]:
import numpy as np
import tensorflow as tf
x_train, y_train, group_train = np.random.normal(size = (10000, 39)), np.random.binomial(1, 0.5, (10000, )), np.random.binomial(1, 0.5, (10000, 2))
x_test, y_test, group_test = np.random.normal(size = (6000, 39)), np.random.binomial(1, 0.5, (6000, )), np.random.binomial(1, 0.5, (6000, 2))

In [4]:
x_train, y_train, group_train = tf.cast(x_train, tf.float32), tf.one_hot(y_train, 2), tf.cast(group_train, tf.int32)
x_test, y_test, group_test = tf.cast(x_test, tf.float32), tf.one_hot(y_test, 2), tf.cast(group_test, tf.int32)
data_train = x_train, y_train, group_train
data_test = x_test, y_test, group_test

In [3]:
import itertools
groups = list(itertools.product([0,1], [0,1]))
groups = [list(x) for x in groups]
groups = tf.cast(groups, tf.int32)

In [4]:
groups 

<tf.Tensor: shape=(4, 2), dtype=int32, numpy=
array([[0, 0],
       [0, 1],
       [1, 0],
       [1, 1]], dtype=int32)>

In [5]:
import setup2
e = setup2.GroupFairness(epoch = 1000)
e.set_graph(classifier_input_shape=(39,), n_groups=4)

In [6]:
e.create_tensorboard()

In [7]:
e.fit(data_train, data_test, groups)

Instructions for updating:
Use tf.identity instead.
Test accuracy for step 500: 0.4975000023841858

Test GAP RMS for 0 and step 500: 0.012841865420341492

Test GAP RMS for 1 and step 500: 0.0238004419952631

Test balanced accuracy for step 500: 0.4988960921764374


Test accuracy for step 750: 0.5005000233650208

Test GAP RMS for 0 and step 750: 0.013324983417987823

Test GAP RMS for 1 and step 750: 0.02386459894478321

Test balanced accuracy for step 750: 0.5005521178245544


{'epsilon': 1.0, 'lr': 0.0001, 'wlr': 1e-05, 'w_reg': 0.01}
Final test accuracy: 0.5011666417121887
Final test GAP RMS for 0: 0.02432774007320404
Final test GAP RMS for 1: 0.019284237176179886
Final test balanced accuracy: 0.5010819435119629




In [8]:
e.classifier.model.save('saved-models/test-model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved-models/test-model/assets


In [2]:
model = tf.keras.models.load_model('saved-models/test-model')

In [3]:
model.summary()

Model: "classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden-layer-1 (Dense)       (None, 10)                400       
_________________________________________________________________
hidden-layer-2 (Dense)       (None, 20)                220       
_________________________________________________________________
logit-layer (Dense)          (None, 2)                 42        
Total params: 662
Trainable params: 662
Non-trainable params: 0
_________________________________________________________________


In [5]:
model(x_train)

<tf.Tensor: shape=(10000, 2), dtype=float32, numpy=
array([[ 0.36286703,  0.6531146 ],
       [-0.15655662,  0.12124513],
       [ 0.5716924 ,  0.51805305],
       ...,
       [ 0.15494733, -0.10417779],
       [ 0.00360449, -0.00113279],
       [-0.5549497 , -0.11485155]], dtype=float32)>

In [4]:
import os
import itertools
reg_w = [10, 20, 30, 40, 50, 60]
wrls = [1e-3, 1e-4, 1e-5, 5e-6, 1e-6]
eps = [0.1, 0.01, 0.001, 0.0001, 0.00001]
seed = 1
l2_reg = 0
lr = 1e-4




for w_reg, wlr, epsilon in itertools.product(reg_w, wrls, eps):
    filename = f'./saved-models/seed-{seed}-lr-{lr}-wlr-{wlr}           -epsilon-{epsilon}-w_reg-{w_reg}-l2_reg-{l2_reg}/*'
    modified_filename = f'./saved-models/seed-{seed}-lr-{lr}-wlr-{wlr}-epsilon-{epsilon}-w_reg-{w_reg}-l2_reg-{l2_reg}/*'
    os.system(f'mv {filename} {modified_filename}')


In [1]:
import numpy as np
import tensorflow as tf
from compas_proc import get_compas_train_test, get_compas_orig
import setup2

In [2]:
seed = 1
x_train, x_test, y_train, y_test, y_sex_train, y_sex_test, y_race_train, y_race_test, feature_names, \
     _, _, _, _ , _, _ = get_compas_train_test(get_compas_orig(), seed=seed)

sex
race


In [3]:
feature_names

['sex',
 'race',
 'priors_count',
 'age_cat=25 to 45',
 'age_cat=Greater than 45',
 'age_cat=Less than 25',
 'c_charge_degree=F',
 'c_charge_degree=M']

In [4]:
x_train

array([[ 1.        ,  0.        , -0.09729493, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.        ,  1.        , -0.30008345, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.        ,  1.        ,  2.53895576, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 1.        ,  0.        ,  1.11943616, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.        ,  0.        ,  0.10549358, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.        ,  0.        , -0.30008345, ...,  1.        ,
         0.        ,  1.        ]])

In [6]:
y_train.astype('int32')

array([1, 1, 1, ..., 1, 0, 1], dtype=int32)

In [7]:
x_train[:, :2]

array([[1., 0.],
       [0., 0.],
       [0., 0.],
       ...,
       [1., 1.],
       [0., 0.],
       [0., 1.]])

In [8]:
y_sex_train

array([1., 0., 0., ..., 1., 0., 0.])

In [9]:
y_race_train

array([0., 0., 0., ..., 1., 0., 1.])

In [10]:
x_train.shape[1]

8

In [1]:
from adult_expt_gender_race import *

sex
race
Instructions for updating:
Use tf.identity instead.


InvalidArgumentError: Invalid reduction dimension (1 for input with 0 dimension(s) [Op:All]

In [3]:
import tensorflow as tf
a = tf.cast([], dtype = tf.bool)
tf.reduce_all(a)

<tf.Tensor: shape=(), dtype=bool, numpy=True>